In [1]:
import pandas as pd

# Load the Excel file
file_path = "Operationalisierung der CVs (1).xlsx"

# Read the Excel file and set the proper header row (row 5, zero-indexed as 4)
df = pd.read_excel(file_path, header=4)

# Rename the columns to their appropriate names based on the first row of the data
df.columns = [
    "Unnamed", "Kategorie", "Oberbegriff", "Unterbegriff_1", 
    "Unterbegriff_2", "Unterbegriff_3", "Beispiele", "Beschreibung"
]

# Drop the unnecessary "Unnamed" column (placeholder for unnamed index column)
df = df.drop(columns=["Unnamed"])

# Reset the index for a clean view
df = df.reset_index(drop=True)

# Reorder columns to make 'Beispiele' the first column
columns = ["Beispiele", "Kategorie", "Oberbegriff", "Unterbegriff_1", 
           "Unterbegriff_2", "Unterbegriff_3", "Beschreibung"]
df = df[columns]

df.drop(index = 0,inplace = True)

set(df.loc[pd.notna(df.Beschreibung)].Beschreibung)

c:\Users\alise\AppData\Local\Programs\Python\Python313\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


{'Abteilungs-/ Departementsleitung sowie deren Stellvertretung',
 'Institutsdirektion & Stellvertretung/ Dekan*innen/ Rektor*innen',
 'Klinikdirektion [Leitung & Stellvertretung], Schulleitung, Chefärzt*innen',
 'Leitung bzw. Management von Forschungsgruppen, Projekten & Laboren',
 'Leitung praxisnaher Forschungszentren',
 'Sektions-/Abteilungs-/Teamleitungen',
 'leitende Ärzt*innen, Oberärzt*innen, Funktionsoberärzt*innen'}

In [2]:
import pandas as pd

# Read the CSV file with proper settings
df = pd.read_csv(
    "Operationalisierung der CVs (1).csv",
    sep=";",  # Specify the correct delimiter
    skip_blank_lines=True,  # Skip empty rows
    quotechar='"',  # Handle quoted fields
    escapechar="\\",  # Handle escape characters for quotes
    on_bad_lines="skip",  # Skip problematic lines
    engine="python"  # Use the Python engine for more flexibility
)

df.drop(index=[0], inplace = True)
df.columns = df.iloc[0]  # Set the first row as the header
df = df[1:]  # Drop the first row

# Reset the index if necessary
df.reset_index(drop=True, inplace=True)

In [3]:
import numpy as np
df_awards = pd.DataFrame(columns=df.columns)
i = 0
for index,row in df.iterrows():
    if pd.notna(row.Beispiele):
        
        examples = [i.strip("\n").strip().replace('"', '') for i in row.Beispiele.split(",")] 
        examples_cleaned = [ex for ex in examples if ex]
        for ex in examples_cleaned:
            df_awards.loc[i,"Beispiele"] = ex 
            df_awards.loc[i,df_awards.columns[0:-1]] = df.loc[index,df_awards.columns[0:-1]]
            i+=1
            

    else:
         df_awards.loc[i,"Beispiele"] = "no example available"
         df_awards.loc[i,df_awards.columns[0:-1]] = df.loc[index,df_awards.columns[0:-1]]
         i+=1

    

In [4]:
df_awards.head()

1,Kategorie,Oberbegriff,Unterbegriff_1,Unterbegriff_2,Beispiele
0,Academic/Research Awards,Awards from scientific societies,NaN,NaN,bruno kisch preis der deutschen gesellschaft f...
1,Academic/Research Awards,Awards from scientific societies,NaN,NaN,bruno kisch research prize
2,Academic/Research Awards,Career Awards,Early Career Awards,PhD,auszeichnung der promotionschrift
3,Academic/Research Awards,Career Awards,Early Career Awards,Further Early Career Awards,early career award 2014 der zeitschrift diskur...
4,Academic/Research Awards,Career Awards,Early Career Awards,Further Early Career Awards,early career award by the leibniz kolleg potsdam


In [5]:
df_awards.to_csv("awards_structure_ref.csv", index=False)

In [6]:
for col in df_awards.columns[:-1]:

    print(f"possible values for {col} : {list(df_awards[col].dropna().unique())}")

possible values for Kategorie : ['Academic/Research Awards', 'Community & Public Engagement Awards', 'No Award', 'Professional [Field] Awards']
possible values for Oberbegriff : ['Awards from scientific societies', 'Career Awards', 'Grants', 'International Collaboration Awards', 'Publications', 'Talks & Presentations', 'Teaching', 'Diversity & Inclusion']
possible values for Unterbegriff_1 : ['Early Career Awards', 'Further Career Awards', 'Fellowships & Scholarships']
possible values for Unterbegriff_2 : ['PhD', 'Further Early Career Awards', 'Habilitation']


In [7]:
import json 

with open("awards_dic_gpt.json","r") as file:
    awards_dict = json.load(file)
awards = []
for key,values in awards_dict.items():
    awards.extend(values)
awards = sorted(list(set(awards)))

In [19]:
index = [i for i in range(0,len(awards),50)]
j = 0
i = index[j]
#awards[i:i+100]

In [79]:
formatted_data = df_awards.apply(lambda row: {row["Beispiele"]:{
    #"string": row["Beispiele"],
    "Kategorie": row["Kategorie"],
    "Oberbegriff": row["Oberbegriff"],
    "Unterbegriff_1": row["Unterbegriff_1"],
    "Unterbegriff_2": row["Unterbegriff_2"]
}}, axis=1).tolist()

In [89]:
df_string = ""
for item in formatted_data:
    df_string += f"{list(item.keys())[0]} : {list(item.values())[0]}\n" 

In [ ]:
import os
import json
from typing import Optional, List
from pydantic import BaseModel, Field#, validator
import openai


# Set your OpenAI API key (or use openai.api_key = "sk-..." directly):
key = "YOUR KEY"
client = openai.OpenAI(api_key=key)

# ---------------------------
# 1) Define possible values
# ---------------------------
KATEGORIE_VALUES = [
    "Academic/Research Awards",
    "Community & Public Engagement Awards",
    "No Award",
    "Professional [Field] Awards"
]

OBERBEGRIFF_VALUES = [
    "Awards from scientific societies",
    "Career Awards",
    "Grants",
    "International Collaboration Awards",
    "Publications",
    "Talks & Presentations",
    "Teaching",
    "Diversity & Inclusion"
]

UNTERBEGRIFF_1_VALUES = [
    "Early Career Awards",
    "Further Career Awards",
    "Fellowships & Scholarships"
]

UNTERBEGRIFF_2_VALUES = [
    "PhD",
    "Further Early Career Awards",
    "Habilitation"
]


# -------------------------------------------------------
# 2) Create a Pydantic model to validate each output row
# -------------------------------------------------------
class AwardPrediction(BaseModel):
    """
    Each instance of AwardPrediction represents the classification
    of a single award string into Kategorie, Oberbegriff, Unterbegriff_1, and Unterbegriff_2.
    """
    
    Kategorie: str = Field(
        ...,
        description=f"One of the values in {KATEGORIE_VALUES}."
    )
    Oberbegriff: Optional[str] = Field(
        None,
        description=f"One of the values in {OBERBEGRIFF_VALUES} or 'NaN'."
    )
    Unterbegriff_1: Optional[str] = Field(
        None,
        description=f"One of the values in {UNTERBEGRIFF_1_VALUES} or 'NaN'."
    )
    Unterbegriff_2: Optional[str] = Field(
        None,
        description=f"One of the values in {UNTERBEGRIFF_2_VALUES} or 'NaN'."
    )

# ---------------------------------
# 3) System prompt with CSV context
# ---------------------------------
CSV_EXAMPLE_TEXT = f"""
[STRING FORMATTED PANDAS DATAFRAME AS AN EXAMPLE]

{df_string}

Remember the valid sets:
Kategorie: [Academic/Research Awards, Community & Public Engagement Awards, No Award, Professional [Field] Awards]
Oberbegriff: [Awards from scientific societies, Career Awards, Grants, International Collaboration Awards, Publications, Talks & Presentations, Teaching, Diversity & Inclusion] or None
Unterbegriff_1: [Early Career Awards, Further Career Awards, Fellowships & Scholarships] or None
Unterbegriff_2: [PhD, Further Early Career Awards, Habilitation] or None

Always return valid JSON with these 4 keys:
{{
  
  "Kategorie": "...",
  "Oberbegriff": "...",
  "Unterbegriff_1": "...",
  "Unterbegriff_2": "..."
}}
If no category fits, use None for those fields except Kategorie which you always have to assign a value a to it. Use internet if you were not sure about an award. 
"""

SYSTEM_PROMPT = f"""
You are a classification model. 
You receive a short award string and must classify it into:
- Kategorie (one of the 4 valid options)
- Oberbegriff (one of the 8 valid options or NaN)
- Unterbegriff_1 (one of the 3 valid options or NaN)
- Unterbegriff_2 (one of the 3 valid options or NaN)

Here is the CSV excerpt illustrating how certain awards were categorized:

{CSV_EXAMPLE_TEXT}

Your goal: For any new input string, return valid JSON with exactly these keys:
  "Kategorie", "Oberbegriff", "Unterbegriff_1", "Unterbegriff_2".
"""


def classify_award_string(text: str,model) -> AwardPrediction:
    """
    Sends one input string to GPT and asks it to produce JSON
    that can be parsed as AwardPrediction. 
    """
    user_message = f"Classify this list of strings:\n{text}\n Return valid JSON for each string in the list."

    response = client.beta.chat.completions.parse(
        model=model,
        temperature=0,
        messages=[
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": user_message}
        ], 
        response_format = AwardPrediction
    )

    content = response.choices[0].message.parsed

    return content

    


def classify_award_list(strings: List[str],model) -> List[AwardPrediction]:
    """
    Classifies a list of award strings by calling GPT for each.
    Returns a list of AwardPrediction objects.
    """
    results = {}
    for s in strings:
        prediction = classify_award_string(text = s,model = model)
        results[s] = prediction.model_dump()
    return results




In [146]:
r = 50 
for i in range(0,len(awards),r): 
    predictions_4o = classify_award_list(awards[i:i+r],model="gpt-4o")


    # Path to the existing JSON file
    file_path = 'awards_structure.json'

    # Step 1: Open and load the existing JSON file
    with open(file_path, 'r', encoding='utf-8') as file:
        current_data = json.load(file)

    # Step 2: Update the current dictionary with the new data
    current_data.update(predictions_4o)

    # Step 3: Save the updated dictionary back to the JSON file
    with open(file_path, 'w', encoding='utf-8') as file:
        json.dump(current_data, file, ensure_ascii=False, indent=4)

    print(f"New data appended successfully! chunk : {i}")



New data appended successfully! chunk : 0
New data appended successfully! chunk : 50
New data appended successfully! chunk : 100
New data appended successfully! chunk : 150
New data appended successfully! chunk : 200
New data appended successfully! chunk : 250
New data appended successfully! chunk : 300
New data appended successfully! chunk : 350
New data appended successfully! chunk : 400
New data appended successfully! chunk : 450
New data appended successfully! chunk : 500
New data appended successfully! chunk : 550
New data appended successfully! chunk : 600
New data appended successfully! chunk : 650
New data appended successfully! chunk : 700
New data appended successfully! chunk : 750
New data appended successfully! chunk : 800
New data appended successfully! chunk : 850
New data appended successfully! chunk : 900
New data appended successfully! chunk : 950
New data appended successfully! chunk : 1000
New data appended successfully! chunk : 1050
New data appended successfully! c

In [6]:
import json 
import pandas as pd 
file_path = 'awards_structure.json'

with open(file_path, 'r', encoding='utf-8') as file:
        current_data = json.load(file)

Check

In [1]:
import pandas as pd 
awards_structured = pd.read_csv("awards_structured.csv")
awards_structured.rename(columns = {'Unnamed: 0': "awards"}, inplace=True)

# Read the CSV file with proper settings
df = pd.read_csv(
    "Operationalisierung der CVs (1).csv",
    sep=";",  # Specify the correct delimiter
    skip_blank_lines=True,  # Skip empty rows
    quotechar='"',  # Handle quoted fields
    escapechar="\\",  # Handle escape characters for quotes
    on_bad_lines="skip",  # Skip problematic lines
    engine="python"  # Use the Python engine for more flexibility
)

df.drop(index=[0], inplace = True)
df.columns = df.iloc[0]  # Set the first row as the header
df = df[1:]  # Drop the first row

# Reset the index if necessary
df.reset_index(drop=True, inplace=True)

In [2]:
subset = ['Kategorie', 'Oberbegriff', 'Unterbegriff_1', 'Unterbegriff_2']

In [3]:
unique_combinations_gpt = awards_structured.drop_duplicates(subset=subset)

In [4]:
unique_combinations = df.drop_duplicates(subset=subset)

In [5]:
df1 = unique_combinations_gpt[subset].sort_values(by='Kategorie', ascending=True, inplace=False)


In [6]:
df2 = unique_combinations[subset].sort_values(by='Kategorie', ascending=True, inplace=False)

In [7]:
difference = pd.merge(df1, df2, how='outer', indicator=True).query('_merge == "left_only"').drop('_merge', axis=1)

In [8]:
difference

,Kategorie,Oberbegriff,Unterbegriff_1,Unterbegriff_2
0,Academic/Research Awards,Awards from scientific societies,Early Career Awards,Further Early Career Awards
1,Academic/Research Awards,Awards from scientific societies,Early Career Awards,NaN
5,Academic/Research Awards,Career Awards,Early Career Awards,NaN
6,Academic/Research Awards,Career Awards,Fellowships & Scholarships,Habilitation
7,Academic/Research Awards,Career Awards,Fellowships & Scholarships,PhD
10,Academic/Research Awards,Career Awards,Further Career Awards,NaN
11,Academic/Research Awards,Career Awards,NaN,NaN
13,Academic/Research Awards,International Collaboration Awards,Fellowships & Scholarships,NaN
16,Academic/Research Awards,Talks & Presentations,Early Career Awards,Further Early Career Awards
19,Academic/Research Awards,NaN,NaN,NaN


In [9]:
difference.fillna("no",inplace=True)
awards_structured.fillna("no",inplace=True)

In [15]:
s = 0
for i in range(len(difference)):
    s+=len(awards_structured[(awards_structured[difference.columns]  == difference.iloc[i]).all(axis=1)])
print(s)

117


In [12]:
dataframes = []

for i in range (len(difference)):
    print(awards_structured[(awards_structured[difference.columns]  == difference.iloc[i]).all(axis=1)].to_string())
    dataframes.append(awards_structured[(awards_structured[difference.columns]  == difference.iloc[i]).all(axis=1)])
    print("\n")



                                                                                    awards                 Kategorie                       Oberbegriff       Unterbegriff_1               Unterbegriff_2
600                               british society for haematology young investigator prize  Academic/Research Awards  Awards from scientific societies  Early Career Awards  Further Early Career Awards
1968  heinz heckhausen jungwissenschaftlerpreis der deutschen gesellschaft für psychologie  Academic/Research Awards  Awards from scientific societies  Early Career Awards  Further Early Career Awards


                                                                                                                                      awards                 Kategorie                       Oberbegriff       Unterbegriff_1 Unterbegriff_2
379   award for young scientists promega biotech ibérica received from the spanish society of biochemistry and molecular biology sebbm spain  Academic/Researc

In [20]:
import numpy as np 
spacer = pd.DataFrame(np.nan, index=[0], columns=dataframes[0].columns)

processed_dataframes = []
for df in dataframes:
    df_with_index = df.reset_index()  # Reset index and keep it as a column
    processed_dataframes.extend([df_with_index])#, spacer])

# Concatenate all DataFrames and drop the last spacer
result = pd.concat(processed_dataframes, ignore_index=True)[:-1]

In [21]:
result

,index,awards,Kategorie,Oberbegriff,Unterbegriff_1,Unterbegriff_2
0,600,british society for haematology young investig...,Academic/Research Awards,Awards from scientific societies,Early Career Awards,Further Early Career Awards
1,1968,heinz heckhausen jungwissenschaftlerpreis der ...,Academic/Research Awards,Awards from scientific societies,Early Career Awards,Further Early Career Awards
2,379,award for young scientists promega biotech ibé...,Academic/Research Awards,Awards from scientific societies,Early Career Awards,no
3,2332,junior award european society of ophthalmic pl...,Academic/Research Awards,Awards from scientific societies,Early Career Awards,no
4,295,aufnahme in die junge akademie,Academic/Research Awards,Career Awards,Early Career Awards,no
...,...,...,...,...,...,...
111,3627,röntgenpreis der deutschen röntgengesellschaft,Professional [Field] Awards,Awards from scientific societies,no,no
112,3823,st jude preis der deutschen gesellschaft für t...,Professional [Field] Awards,Awards from scientific societies,no,no
113,3838,staude pfannenstiel preis der norddeutschen ge...,Professional [Field] Awards,Awards from scientific societies,no,no
114,3953,thannhauser preis der deutschen gesellschaft f...,Professional [Field] Awards,Awards from scientific societies,no,no


In [ ]:
result.replace("no",np.nan).to_excel("awards_new_categories.xlsx",index = False)

: 

In [38]:
df1.fillna("no",inplace=True)

dataframes = []

for i in range (len(df1)):
    #print(awards_structured[(awards_structured[df1.columns]  == df1.iloc[i]).all(axis=1)].to_string())
    dataframes.append(awards_structured[(awards_structured[df1.columns]  == df1.iloc[i]).all(axis=1)])
    #print("\n")

import numpy as np 
spacer = pd.DataFrame(np.nan, index=[0], columns=dataframes[0].columns)

processed_dataframes = []
for df in dataframes:
    df_with_index = df.reset_index()  # Reset index and keep it as a column
    processed_dataframes.extend([df_with_index, spacer])

# Concatenate all DataFrames and drop the last spacer
result = pd.concat(processed_dataframes, ignore_index=True)[:-1]

result.replace("no",np.nan).to_excel("awards_all_categories.xlsx",index = False)

In [165]:
awards_structured.loc[(awards_structured[difference.columns]  == difference.iloc[2]).all(axis=1), "Unterbegriff_2"] = "Further Early Career Awards"

# correcting the categories made by gpt 

In [43]:
import pandas as pd 

df_newcat = pd.read_excel("awards_new_categories - Copy.xlsx")
df_newcat = df_newcat.dropna(how="all")
df_newcat["index"] = df_newcat["index"].astype(int)

In [44]:
df_newcat

,index,awards,Kategorie,Oberbegriff,Unterbegriff_1,Unterbegriff_2
0,600,british society for haematology young investig...,Academic/Research Awards,Awards from scientific societies,NaN,NaN
1,1968,heinz heckhausen jungwissenschaftlerpreis der ...,Academic/Research Awards,Awards from scientific societies,NaN,NaN
3,379,award for young scientists promega biotech ibé...,Academic/Research Awards,Awards from scientific societies,NaN,NaN
4,2332,junior award european society of ophthalmic pl...,Academic/Research Awards,Awards from scientific societies,NaN,NaN
6,295,aufnahme in die junge akademie,Academic/Research Awards,Career Awards,Early Career Awards,Further Early Career Awards
...,...,...,...,...,...,...
125,3823,st jude preis der deutschen gesellschaft für t...,Professional [Field] Awards,NaN,NaN,NaN
126,3838,staude pfannenstiel preis der norddeutschen ge...,Professional [Field] Awards,NaN,NaN,NaN
127,3953,thannhauser preis der deutschen gesellschaft f...,Professional [Field] Awards,NaN,NaN,NaN
128,4168,von langenbeck preis der deutschen gesellschaf...,Professional [Field] Awards,NaN,NaN,NaN


In [45]:
awards_structured = pd.read_csv("awards_structured.csv")
awards_structured = awards_structured.rename(columns={"Unnamed: 0":"awards"})

In [67]:
awards_structured.loc[index,"awards"]==df_newcat.loc[df_newcat["index"]==index,"awards"].values[0]

True

In [70]:
newcat_idx = df_newcat["index"]
for index in newcat_idx:
    values_to_assign = df_newcat.loc[df_newcat["index"] == index, ["Kategorie", "Oberbegriff", "Unterbegriff_1", "Unterbegriff_2"]].iloc[0]
    awards_structured.loc[index, ["Kategorie", "Oberbegriff", "Unterbegriff_1", "Unterbegriff_2"]] = values_to_assign.values




In [77]:
awards_structured.to_csv("awards_structured.csv",index=False)